# making an aggregate master dataframe for baseline model
this notebook is going to perform what 07-Data_merging was hoping to accomplish but in a new light. 
instead of trying to make one master dataframe that i would adjust all data for each model to, I will instead make a master database for each type of data organization. for my first pass, i will work to establish a baseline model by using:

the single "worst", or value that most indicates poor clinical outcomes, for each variable so each variable only has one row per patient. 

from my notes:

- Step1: establish a baseline (ie train model initially), using an obvious baseline: last valid mesurement of a particular variable.   Will hope that it doesn’t perform too good or too bad. Second would be to pick an aggregate within a time window (over 3 days, or of each day, ie can change graunlarity).

- Next try temporal trend, maybe vector autoregression. 

So my first step would be to pick either the last recorded value for each variable or the ‘worst’ value, or ones that we might expect to indicate poor outcome (NEED TO CHOOSE)


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/01102018_final_pt_df2_v.csv') , index_col=0) #only for patients with minimum vitals
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 119 ms


In [100]:
#adding this in fow now, will eventually want to switch this to the 03-clinical_variables_final
import psycopg2
import getpass
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

Password: ·····


time: 2.3 s


In [ ]:
#import all clinical variables
##ensure they are the versions with UOM

In [3]:
#reading in all of my data that is not limited to 72 hour time window between t_0 and t+72
date= '27082018' 

ventcategory_df=pd.read_csv(Path(
    wd+'/data/raw/csv/%s_ventcategory_df.csv' %(date)), index_col=0)

vaso_dose_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date)), index_col=0)


echodata_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date)), index_col=0)
labs_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_labs_all_nosummary_72.csv' %(date)), index_col=0)


weightfirstday_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date)), index_col=0)
heightfirstday_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_heightfirstday_df.csv' %(date)), index_col=0)

vitals_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date)), index_col=0)
uti_all_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date)), index_col=0)
bg_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date)), index_col=0)

rrt_merged_allpt_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_rrt_merged_allpt_df.csv' %(date)), index_col=0)
gcs72_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_gcs72_df.csv' %(date)), index_col=0)

sofa_df_72=pd.read_csv(Path(
    wd+'/data/raw/csv/%s_sofa_df_72.csv' %(date)), index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 15.7 s


In [4]:
def df_subset(df):
    try:
        df = df.loc[df.loc[:,'icustay_id'].isin(icustay_id),:]
    except KeyError or NameError:
        try:
            df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id),:]
        except KeyError or NameError:
            df = df.loc[df.loc[:,'subject_id'].isin(patients),:]
    return(df)

time: 5.63 ms


In [5]:
# subsetting each dataframe to only patients in final_patients_df2_v:
dataframe_list= [
    ventcategory_df, vaso_dose_72, echodata_72,
    labs_all_nosummary_72, weightfirstday_df,
    heightfirstday_df, vitals_all_nosummary_72,
    uti_all_72, bg_all_nosummary_72,
    rrt_merged_allpt_df, gcs72_df, sofa_df_72
]

(
ventcategory_df, vaso_dose_72, echodata_72,
labs_all_nosummary_72, weightfirstday_df,
heightfirstday_df, vitals_all_nosummary_72,
uti_all_72, bg_all_nosummary_72,
rrt_merged_allpt_df, gcs72_df, sofa_df_72
) = (df_subset(df) for df in dataframe_list)

time: 1.11 s


10/5/18
# Standardizing Columns in df to merge together

10/22/18:
unit importing notebook has adjusted units, need to go through each individually and see which still needs work.

### list of columns in master df:
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- dtype

### Vaso_dose

In [81]:
#vaso_dose_72.head()
vaso_dose_72= vaso_dose_72.rename(
    columns={'starttime':'charttime','label':'vaso_type'}
        )#.drop(labels='endtime', axis=1) 
#renaming starttime to charttime and dropping endtime

vaso_dose_72['uom']= 'mg'

time: 20.6 ms


In [7]:
vaso_dose_72['rate_uom'].unique()

array(['mcg/kg/min', 'units/hour', 'mcgkgmin', 'Umin', 'Uhr'],
      dtype=object)

time: 5.62 ms


In [10]:
len(vaso_dose_72)

53786

time: 2.26 ms


In [11]:
vaso_dose_72= vaso_dose_72.loc[~vaso_dose_72.loc[:,'rate_uom'].isin(['Uhr','units/hour']),:]

time: 9.17 ms


## formatting the vaso_dose df to meet the master df standard:

In [89]:
# for element in list(vaso_dose_72['vaso_type'].unique()):
#     vaso_dose_72.loc[vaso_dose_72.loc[:,'vaso_type']==str(element),'vaso_type']= '%s_rate'%(element)
    
# vaso_dose_72= vaso_dose_72.rename(
#     columns={'vaso_type':'label','vaso_rate':'value'}).drop(labels='vaso_amount', axis=1)

AttributeError: 'dict' object has no attribute 'drop'

time: 74.6 ms


# labs_all_nosummary_72
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [179]:
#labs_all_nosummary_72
labs_all_nosummary_72.head(20)

,subject_id,hadm_id,icustay_id,charttime,label,value,t_0,delta,uom,endtime
0,9,150750,220597,2149-11-14 07:47:00,SODIUM,137.0,2149-11-13,1 days 07:47:00.000000000,mEq/L,NaN
1,9,150750,220597,2149-11-13 03:20:00,SODIUM,130.0,2149-11-13,0 days 03:20:00.000000000,mEq/L,NaN
2,9,150750,220597,2149-11-13 16:04:00,SODIUM,129.0,2149-11-13,0 days 16:04:00.000000000,mEq/L,NaN
3,9,150750,220597,2149-11-13 19:30:00,SODIUM,131.0,2149-11-13,0 days 19:30:00.000000000,mEq/L,NaN
4,9,150750,220597,2149-11-14 01:48:00,SODIUM,133.0,2149-11-13,1 days 01:48:00.000000000,mEq/L,NaN
5,9,150750,220597,2149-11-14 06:00:00,SODIUM,137.0,2149-11-13,1 days 06:00:00.000000000,mEq/L,NaN
6,19,109235,273430,2108-08-06 01:08:00,SODIUM,138.0,2108-08-06,0 days 01:08:00.000000000,mEq/L,NaN
7,19,109235,273430,2108-08-07 05:07:00,SODIUM,139.0,2108-08-06,1 days 05:07:00.000000000,mEq/L,NaN
8,19,109235,273430,2108-08-08 05:20:00,SODIUM,141.0,2108-08-06,2 days 05:20:00.000000000,mEq/L,NaN
9,21,111970,216859,2135-02-05 04:48:00,SODIUM,134.0,2135-02-05,0 days 04:48:00.000000000,mEq/L,NaN


time: 17.8 ms


#### adding in UOM into labs

In [107]:
# #importing unit of mesurements:
# def uom_sql_import(file_path):
#     if isinstance(file_path, str)== True:
#         f = open(Path(file_path), 'r')
#     else:
#         f = open(Path(str(file_path)), 'r')

#     SQL = open(file_path,'r').read()
#     SQL_df= pd.read_sql_query(SQL,conn)   
#     return(SQL_df)

time: 4.95 ms


In [129]:
lab_uom= uom_sql_import(Path(wd+'/data/raw/clinical_var_sql/unit_of_mesurement/labs_uom.sql'))


time: 22.2 s


In [130]:
lab_uom

,uom,label
0,g/dL,ALBUMIN
1,mEq/L,ANION GAP
2,%,BANDS
3,mEq/L,BICARBONATE
4,mg/dL,BILIRUBIN
5,mg/dL,BUN
6,mEq/L,CHLORIDE
7,mg/dL,CREATININE
8,mg/dL,GLUCOSE
9,%,HEMATOCRIT


time: 7.97 ms


In [124]:
labs_all_nosummary_72 = pd.merge(labs_all_nosummary_72, lab_uom, left_on='label', right_on='label')

time: 365 ms


In [131]:
del(lab_uom)

time: 704 µs


In [137]:
labs_all_nosummary_72.head()

,subject_id,hadm_id,icustay_id,charttime,label,value,t_0,delta,uom,endtime
0,9,150750,220597,2149-11-14 07:47:00,SODIUM,137.0,2149-11-13,1 days 07:47:00.000000000,mEq/L,NaN
1,9,150750,220597,2149-11-13 03:20:00,SODIUM,130.0,2149-11-13,0 days 03:20:00.000000000,mEq/L,NaN
2,9,150750,220597,2149-11-13 16:04:00,SODIUM,129.0,2149-11-13,0 days 16:04:00.000000000,mEq/L,NaN
3,9,150750,220597,2149-11-13 19:30:00,SODIUM,131.0,2149-11-13,0 days 19:30:00.000000000,mEq/L,NaN
4,9,150750,220597,2149-11-14 01:48:00,SODIUM,133.0,2149-11-13,1 days 01:48:00.000000000,mEq/L,NaN


time: 13.1 ms


## formatting the labs_all_nosummary_72 df to meet the master df standard:


In [134]:
labs_all_nosummary_72['endtime']= np.nan #adding in null endtimes

time: 6.85 ms


In [136]:
labs_all_nosummary_72= labs_all_nosummary_72.rename(
    columns={'valuenum':'value'}) #changing valuenum to value

time: 516 ms


In [ ]:
#missing days, may or may not keep this

In [139]:
labs_all_nosummary_72.head()

,subject_id,hadm_id,icustay_id,charttime,label,value,t_0,delta,uom,endtime
0,9,150750,220597,2149-11-14 07:47:00,SODIUM,137.0,2149-11-13,1 days 07:47:00.000000000,mEq/L,NaN
1,9,150750,220597,2149-11-13 03:20:00,SODIUM,130.0,2149-11-13,0 days 03:20:00.000000000,mEq/L,NaN
2,9,150750,220597,2149-11-13 16:04:00,SODIUM,129.0,2149-11-13,0 days 16:04:00.000000000,mEq/L,NaN
3,9,150750,220597,2149-11-13 19:30:00,SODIUM,131.0,2149-11-13,0 days 19:30:00.000000000,mEq/L,NaN
4,9,150750,220597,2149-11-14 01:48:00,SODIUM,133.0,2149-11-13,1 days 01:48:00.000000000,mEq/L,NaN


time: 21.1 ms


# ventcategory_df_melt
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [140]:
ventcategory_df_melt = pd.melt(ventcategory_df, id_vars=['icustay_id','t_0'], var_name='label')
ventcategory_df_melt.head()

,icustay_id,t_0,label,value
0,267059.0,2182-11-12,vent_day1,Mech
1,278729.0,2159-11-08,vent_day1,Oxygen
2,219670.0,2190-08-01,vent_day1,Oxygen
3,295185.0,2154-11-11,vent_day1,Oxygen
4,211472.0,2193-12-11,vent_day1,None


time: 23.2 ms


In [162]:
#adding in charttime

ventcategory_df_melt.loc[ventcategory_df_melt.loc[:,'label']=='vent_day1','charttime']=(
    pd.to_datetime(
        ventcategory_df_melt.loc[ventcategory_df_melt.loc[:,'label']=='vent_day1','t_0'])
            +pd.Timedelta(days=1))

ventcategory_df_melt.loc[ventcategory_df_melt.loc[:,'label']=='vent_day2','charttime']=(
    pd.to_datetime(
        ventcategory_df_melt.loc[ventcategory_df_melt.loc[:,'label']=='vent_day2','t_0'])
            +pd.Timedelta(days=2))

ventcategory_df_melt.loc[ventcategory_df_melt.loc[:,'label']=='vent_day3','charttime']=(
    pd.to_datetime(
        ventcategory_df_melt.loc[ventcategory_df_melt.loc[:,'label']=='vent_day3','t_0'])
            +pd.Timedelta(days=3))

ventcategory_df_melt['charttime']= pd.to_datetime(ventcategory_df_melt['charttime'])

time: 113 ms


In [165]:
ventcategory_df_melt.head()

,icustay_id,t_0,label,value,uom,frequency,dtype,endtime,charttime,days
0,267059.0,2182-11-12,vent_day1,Mech,category,once_per_day,factor,NaN,2182-11-13,1.0
1,278729.0,2159-11-08,vent_day1,Oxygen,category,once_per_day,factor,NaN,2159-11-09,1.0
2,219670.0,2190-08-01,vent_day1,Oxygen,category,once_per_day,factor,NaN,2190-08-02,1.0
3,295185.0,2154-11-11,vent_day1,Oxygen,category,once_per_day,factor,NaN,2154-11-12,1.0
4,211472.0,2193-12-11,vent_day1,None,category,once_per_day,factor,NaN,2193-12-12,1.0


time: 13 ms


# weightfirstday_df_melt
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [166]:
weightfirstday_df_melt= pd.melt(weightfirstday_df, id_vars='icustay_id', var_name='label')
weightfirstday_df_melt.head()

,icustay_id,label,value
0,200012,weight,51.2
1,200014,weight,62.0
2,200019,weight,65.0
3,200030,weight,113.6
4,200033,weight,74.0


time: 15 ms


In [167]:
weightfirstday_df_melt['uom']='category'
weightfirstday_df_melt['frequency']= 'once_per_stay'
weightfirstday_df_melt['dtype']= 'float'
weightfirstday_df_melt['endtime']=np.NaN
weightfirstday_df_melt['charttime']=np.NaN
weightfirstday_df_melt['days']=1.0
weightfirstday_df_melt['delta']= np.NaN


time: 12.5 ms


In [169]:
weightfirstday_df_melt= (
        pd.merge(weightfirstday_df_melt, 
         final_pt_df2[['icustay_id','hadm_id','t_0']], 
         left_on='icustay_id',
         right_on='icustay_id')
)

time: 58.4 ms


In [171]:
weightfirstday_df_melt.head()

,icustay_id,label,value,uom,frequency,dtype,endtime,charttime,days,delta,hadm_id,t_0
0,200012,weight,51.200000,category,once_per_stay,float,NaN,NaN,1.0,NaN,177527,2153-12-23
1,200012,weight_admit,51.200001,category,once_per_stay,float,NaN,NaN,1.0,NaN,177527,2153-12-23
2,200012,weight_daily,51.200001,category,once_per_stay,float,NaN,NaN,1.0,NaN,177527,2153-12-23
3,200012,weight_echoinhosp,NaN,category,once_per_stay,float,NaN,NaN,1.0,NaN,177527,2153-12-23
4,200012,weight_echoprehosp,NaN,category,once_per_stay,float,NaN,NaN,1.0,NaN,177527,2153-12-23


time: 14.7 ms


# heightfirstday_df_melt
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [ ]:
heightfirstday_df_melt= pd.melt(heightfirstday_df, id_vars='icustay_id', var_name='label')
heightfirstday_df_melt.head()

# vitals_all_nosummary_nonull
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [ ]:
vitals_all_nosummary_nonull= vitals_all_nosummary_72.loc[vitals_all_nosummary_72['vitalid'].notnull(),:]#.count() #6930 NULL values
vitals_all_nosummary_nonull.head()

# uti_all_72_melt_notnull
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [172]:
uti_all_72_melt= pd.melt(
    uti_all_72[['hadm_id','charttime','value','valuenum','valueuom','label','fluid']],
    id_vars=['hadm_id','charttime'], var_name='label')

uti_all_72_melt_notnull= uti_all_72_melt[uti_all_72_melt['value'].notnull()] #1320954

uti_all_72_melt_notnull= uti_all_72_melt_notnull.drop_duplicates(subset=['hadm_id','label','charttime']) #drops the duplicated rows
#n=7796 rows

time: 541 ms


In [177]:
uti_all_72.head()

,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,label,fluid,category,loinc_code,icd9_code,t_0,delta
60,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,99831,2131-05-21,0 days 01:33:00.000000000
61,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,99811,2131-05-21,0 days 01:33:00.000000000
62,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,41511,2131-05-21,0 days 01:33:00.000000000
63,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,4538,2131-05-21,0 days 01:33:00.000000000
64,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,99672,2131-05-21,0 days 01:33:00.000000000
65,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,496,2131-05-21,0 days 01:33:00.000000000
66,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,41401,2131-05-21,0 days 01:33:00.000000000
67,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,V4581,2131-05-21,0 days 01:33:00.000000000
68,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,4019,2131-05-21,0 days 01:33:00.000000000
69,36,122659.0,51486,2131-05-21 01:33:00,NEG,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,60000,2131-05-21,0 days 01:33:00.000000000


time: 60.9 ms


In [174]:
uti_all_72_melt_notnull.head()


,hadm_id,charttime,label,value
0,122659.0,2131-05-21 01:33:00,value,NEG
24,128652.0,2108-08-29 07:50:00,value,NEG
40,129635.0,2160-11-02 08:55:00,value,NEG
48,176332.0,2118-07-15 19:00:00,value,NEG
66,139061.0,2141-09-11 09:15:00,value,MOD


time: 8.56 ms


In [175]:
uti_all_72_melt_notnull['value'].unique()

array(['NEG', 'MOD', 'TR', 'SM', 'LG', 'SMALL', ' ', 'Leukocytes',
       'Nitrite', 'Urine'], dtype=object)

time: 6.47 ms


In [176]:
uti_all_72_melt_notnull['label'].unique()

array(['value', 'label', 'fluid'], dtype=object)

time: 3.27 ms


# bg_all_nosummary_72
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [ ]:
#na values- removing all null values from value.
bg_all_nosummary_72= bg_all_nosummary_72.loc[bg_all_nosummary_72['value'].notnull(),:]
bg_all_nosummary_72.head()

# rrt_merged_allpt_df_notnull
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [ ]:
#na values
rrt_merged_allpt_df_notnull = rrt_merged_allpt_df.loc[rrt_merged_allpt_df['rrt'].notnull(),:]
rrt_merged_allpt_df_notnull

# gcs72_df_melt_notnull
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [ ]:
#need to reshape the dataframe
gcs72_df_melt = pd.melt(gcs72_df, id_vars=['subject_id','hadm_id','icustay_id','t_0','ICU_admit','admit_plus_day','delta'], var_name='label')

gcs72_df_melt_notnull= gcs72_df_melt[gcs72_df_melt['value'].notnull()] #62772 ->62650
gcs72_df_melt_notnull.head()

# sofa_df_melt_notnull
- subject_id
- hadm_id
- icustay_id
- charttime: starttime/charttime.
- endtime: going to be null for any value listed with only a charttime. 
- label
- value
- units?
- t_0
- delta
- days
- frequency
- dtype

In [ ]:
#need to reshape the dataframe
sofa_df_melt = pd.melt(sofa_df_72, id_vars=['subject_id','hadm_id','icustay_id','t_0','day','ICU_admit','approx_charttime','floor_charttime'], var_name='label')

sofa_df_melt_notnull= sofa_df_melt[sofa_df_melt['value'].notnull()] #606651 ->302459
sofa_df_melt_notnull.head()